# Pipeline for ORCA-CLEAN Deep Denoizing Network

**Requirements** - In order to run this notebook, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription - [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace with computer cluster - [Configure workspace](../../configuration.ipynb)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](../../../README.md) - check the getting started section

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1 Import the required libraries

In [ ]:
import datetime

from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential, EnvironmentCredential, AzureCliCredential

from azure.ai.ml import MLClient, Input, Output
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import load_component
from azure.ai.ml.entities import AmlCompute, Environment, Model, Data
from azure.ai.ml.constants import AssetTypes
import os

## 1.2 Configure credential

We are using `DefaultAzureCredential` to get access to workspace. 
`DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios. 

Reference for more available credentials if it does not work for you: [configure credential example](../../configuration.ipynb), [azure-identity reference doc](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

In [ ]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential(tenant_id="16b3c013-d300-468d-ac64-7eda0820b6d3")

## 1.3 Get a handle to the workspace

We use config file to connect to a workspace. The Azure ML workspace should be configured with computer cluster. [Check this notebook for configure a workspace](../../configuration.ipynb)

In [ ]:
# Get a handle to workspace
ml_client = MLClient.from_config(
    credential=credential, 
    # tenant_id="16b3c013-d300-468d-ac64-7eda0820b6d3", # needed to sepcify the tenant to make it work in my sub
    )


compute_instance_name = "whalesdenoisinggpu2"
try:
    print(ml_client.compute.get(compute_instance_name))
except Exception as ex:
    compute_instance = AmlCompute(
        name=compute_instance_name,
        type="amlcompute",
        size="Standard_NC24ads_A100_v4",
        # Minimum running nodes when there is no job running
        min_instances=0,
        # Nodes in cluster
        max_instances=4,
        # How many seconds will the node running after the job termination
        idle_time_before_scale_down=180,
        # Dedicated or LowPriority. The latter is cheaper but there is a chance of job termination
        tier="Dedicated",
    )
    ml_client.begin_create_or_update(compute_instance).result()
    print(f"Created new compute instance: {ml_client.compute.get(compute_instance_name)}")

## 1.4 Create a custom environment

In [ ]:
custom_env_name = "whales-denoising-env"
version = "1.8"
dependencies_dir = "./dependencies"

try:
    pipeline_job_env = ml_client.environments.get(name=custom_env_name, version=version)
    print(
        f"Environment with name {pipeline_job_env.name} is registered to workspace, the environment version is {pipeline_job_env.version}"
    )
except:
    pipeline_job_env = Environment(
        name=custom_env_name,
        description="Custom environment for running ORCA-CLEAN pipeline",
        conda_file=os.path.join(dependencies_dir, "conda.yaml"),
        image="mcr.microsoft.com/azureml/curated/acpt-pytorch-2.0-cuda11.7:17", #"mcr.microsoft.com/azureml/openmpi3.1.2-cuda10.1-cudnn7-ubuntu18.04", #"mcr.microsoft.com/azureml/curated/acpt-pytorch-1.13-cuda11.7:18",
        version=version,
    )

    pipeline_job_env = ml_client.environments.create_or_update(pipeline_job_env)
    print(
        f"Environment with name {pipeline_job_env.name} is registered to workspace, the environment version is {pipeline_job_env.version}"
    )

## 1.5 Create the Data asset objects

In [ ]:
whales_data_name = "whales_data"
noise_train_data_name = "noise_train_data"
noise_test_data_name = "noise_test_data"
noise_val_data_name = "noise_val_data"
version = "2"

In [ ]:
path = "/mnt/humpbackwhales/data/denoising_data/whales_data_v2"
whales_data_name = "whales_data"

try:
    whales_data = ml_client.data.get(whales_data_name, version=version)
except:
    whales_data = Data(
        path=path,
        type=AssetTypes.URI_FOLDER,
        description="whales vocalizations of all types: moans, barks, creaks, etc.",
        name=whales_data_name,
        version=version,
    )
    ml_client.data.create_or_update(whales_data)

In [ ]:
path = "/mnt/humpbackwhales/data/denoising_data/noise_train_v2"
noise_train_data_name = "noise_train_data"

try:
    noise_train_data = ml_client.data.get(noise_train_data_name, version=version)
except:
    noise_train_data = Data(
        path=path,
        type=AssetTypes.URI_FOLDER,
        description="5-sec segments of only-noise retrieved from raw hydrophone recordings",
        name=noise_train_data_name,
        version=version,
    )
    ml_client.data.create_or_update(noise_train_data)

In [ ]:
path = "/mnt/humpbackwhales/data/denoising_data/noise_test_v2"
noise_test_data_name = "noise_test_data"

try:
    noise_test_data = ml_client.data.get(noise_test_data_name, version=version)
except:
    noise_test_data = Data(
        path=path,
        type=AssetTypes.URI_FOLDER,
        description="5-sec segments of only-noise retrieved from raw hydrophone recordings",
        name=noise_test_data_name,
        version=version,
    )
    ml_client.data.create_or_update(noise_test_data)

In [ ]:
path = "/mnt/humpbackwhales/data/denoising_data/noise_val_v2"
noise_val_data_name = "noise_val_data"

try:
    noise_val_data = ml_client.data.get(noise_val_data_name, version=version)
except:
    noise_val_data = Data(
        path=path,
        type=AssetTypes.URI_FOLDER,
        description="5-sec segments of only-noise retrieved from raw hydrophone recordings",
        name=noise_val_data_name,
        version=version,
    )
    ml_client.data.create_or_update(noise_val_data)

In [ ]:
path = "/mnt/humpbackwhales/denoising/outputs"
denoising_training_outputs_name = "denoising_training_outputs"

try:
    denoising_training_outputs = ml_client.data.get(denoising_training_outputs_name, version=version)
except:
    denoising_training_outputs = Data(
        path=path,
        type=AssetTypes.URI_FOLDER,
        description="results of deep denoising network's training",
        name=denoising_training_outputs_name,
        version=version,
    )
    ml_client.data.create_or_update(denoising_training_outputs)

In [ ]:
default_datastore_name = "workspaceblobstore"  # workspaceblobstore, workspacefilestore

# 2. Define and create components into workspace
## 2.1 Load components from YAML

In [ ]:
parent_dir = "."
train_model = load_component(path=parent_dir + "/train_model.yml")
#score_data = load_component(source=parent_dir + "/score_data.yml")
#eval_model = load_component(source=parent_dir + "/eval_model.yml")

## 2.2 Inspect loaded component

# 3. Pipeline job
## 3.1 Build pipeline

In [ ]:
# Construct pipeline
@pipeline()
def pipeline_with_components_from_yaml(
    whales_data_input,
    noise_train_data_input,
    noise_val_data_input,
    noise_test_data_input,
    training_outputs,
    max_train_epochs,
    
):
    """Pipeline with components defined via yaml."""
    train_component = train_model(
        whales_data=whales_data_input,
        noise_train_data=noise_train_data_input,
        noise_val_data=noise_val_data_input,
        noise_test_data=noise_test_data_input,
        max_train_epochs=max_train_epochs,
        model_dir="models",
        log_dir="logs",
        checkpoint_dir="model_checkpoints",
        cache_dir="cache",
        summary_dir="summary",
    )
    
    #mounted_output_dir = "azureml://datastores/workspaceblobstore/paths/azureml/denoising/ML_pipeline"
    
    # train_component.outputs.model_dir = denoising_training_outputs.path + "/models/"
    # train_component.outputs.log_dir = denoising_training_outputs.path + "/logs/"
    # train_component.outputs.checkpoint_dir = denoising_training_outputs.path + "/model_checkpoints/"
    # train_component.outputs.cache_dir = denoising_training_outputs.path + "/cache/"
    # train_component.outputs.summary_dir = denoising_training_outputs.path + "/summary/"

    train_component.inputs.model_dir = "models"
    train_component.inputs.log_dir = "logs"
    train_component.inputs.checkpoint_dir = "model_checkpoints"
    train_component.inputs.cache_dir = "cache"
    train_component.inputs.summary_dir = "summary"

    train_component.outputs.training_output = Output(type="uri_folder", path=training_outputs, mode="rw_mount")

whales_data_name = "whales_data"
noise_train_data_name = "noise_train_data"
noise_test_data_name = "noise_test_data"
noise_val_data_name = "noise_val_data"

pipeline_job = pipeline_with_components_from_yaml(
    whales_data_input=Input(type="uri_folder", path=whales_data.path),
    noise_train_data_input=Input(type="uri_folder", path=noise_train_data.path),
    noise_val_data_input=Input(type="uri_folder", path=noise_val_data.path),
    noise_test_data_input=Input(type="uri_folder", path=noise_test_data.path),
    training_outputs=denoising_training_outputs.path,
    max_train_epochs=2,
)

# set pipeline level compute
pipeline_job.settings.default_compute = compute_instance_name

In [ ]:
# Inspect built pipeline
print(pipeline_job.outputs)

## 3.2 Submit pipeline job

In [ ]:
# Submit pipeline job to workspace
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="whales_denoising"
)
pipeline_job

In [ ]:
# Wait until the job completes
ml_client.jobs.stream(pipeline_job.name)

## 3.3 Register a model

In [ ]:
run_model = Model(
    path="azureml://jobs/{}/outputs/artifacts/paths/model/".format(pipeline_job.name),
    name="orca-clean-model-v1",
    description="Model trained with 5 epochs, whales_data_v2 and noise_train/test/val_v2.",
    type=AssetTypes.MLFLOW_MODEL
)

ml_client.models.create_or_update(run_model)

## 3.4 Create a batch endpoint

In [ ]:
from azure.ai.ml.entities import BatchEndpoint, ModelBatchDeployment, PipelineComponentBatchDeployment

endpoint_name="whales-denoising-batch"

endpoint = BatchEndpoint(
    name=endpoint_name,
    description="Endpoint for pipeline deployments",
)

ml_client.batch_endpoints.begin_create_or_update(endpoint).result()

endpoint = ml_client.batch_endpoints.get(name=endpoint_name)
print(endpoint)

## 3.5 Deploy the pipeline job

In [ ]:
deployment = PipelineComponentBatchDeployment(
    name="whales-denoising-batch-from-job",
    description="This deployment is created from a pipeline job.",
    endpoint_name=endpoint.name,
    job_definition=pipeline_job,
    settings={
        "default_compute": compute_instance_name,
        "continue_on_step_failure": False
    }
)

In [ ]:
ml_client.batch_deployments.begin_create_or_update(deployment).result()

In [ ]:
endpoint = ml_client.batch_endpoints.get(endpoint.name)
endpoint.defaults.deployment_name = deployment.name
ml_client.batch_endpoints.begin_create_or_update(endpoint).result()

## 3.6 Test the deployment

In [ ]:
job = ml_client.batch_endpoints.invoke(
    endpoint_name=endpoint.name, 
)

In [ ]:
ml_client.jobs.get(name=job.name)